<a href="https://colab.research.google.com/github/Hieubui1103/Construction_/blob/main/Florence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
import torch
%matplotlib inline

In [ ]:
model_id = 'microsoft/Florence-2-large-ft'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
%cd /content
!mkdir temp_clone
%cd temp_clone

# Sparse-checkout from here
!git clone --filter=blob:none --no-checkout https://github.com/Hieubui1103/Construction_.git
%cd Construction_
!git sparse-checkout init --cone
!git sparse-checkout set Images
!git checkout main

# get back to the main directory
%cd ../../

In [ ]:
import os
import shutil
import re


# Path to the folder containing 1000 images
image_folder_action = "temp_clone/Construction_/Images/Action_cons_files"
output_folder_action = "images_action"

image_folder_emotion = "temp_clone/Construction_/Images/Emotion_cons_files"
output_folder_emotion = "images_emotion"

# Create output folders if they don't exist
os.makedirs(output_folder_action, exist_ok=True)
os.makedirs(output_folder_emotion, exist_ok=True)



In [ ]:
def create_batches(image_folder, output_folder):
    """
    Create 5 batches of images from the specified folder and copy them into separate folders.
    """
    # Create 5 batch folders
    batch_size = 200
    for batch_num in range(1, 6):
        batch_folder = os.path.join(output_folder, f"batch_{batch_num}")
        os.makedirs(batch_folder, exist_ok=True)

    # Get all image files and sort them numerically
    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])

    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else -1

    image_files.sort(key=extract_number)

    # Distribute images into batches
    for i, file in enumerate(image_files):
        batch_num = (i // batch_size) + 1
        if batch_num > 5:  # Ensure no more than 5 batches
            break
        source_path = os.path.join(image_folder, file)
        destination_path = os.path.join(output_folder, f"batch_{batch_num}", file)
        shutil.copy(source_path, destination_path)

    print("Images have been copied into 5 batches.")


# Create batches for action images
create_batches(image_folder_action, output_folder_action)

# Create batches for emotion images
create_batches(image_folder_emotion, output_folder_emotion)

In [ ]:
import os

# Define the directory
image_dir_batch1 = "images_action/batch_1"
image_dir_batch2 = "images_action/batch_2"
image_dir_batch3 = "images_action/batch_3"
image_dir_batch4 = "images_action/batch_4"
image_dir_batch5 = "images_action/batch_5"

# Get all files in the directory and sort them
def get_sorted_files(image_dir):
  image_files = sorted(
      [os.path.join(image_dir, file) for file in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, file))],
      key=lambda x: int(''.join(filter(str.isdigit, os.path.basename(x)))) if any(c.isdigit() for c in os.path.basename(x)) else x
  )
  return image_files

print(get_sorted_files(image_dir_batch3))  # Prints a sorted list of image file paths


In [ ]:
import os

# Define the directory
emotion_dir_batch1 = "images_emotion/batch_1"
emotion_dir_batch2 = "images_emotion/batch_2"
emotion_dir_batch3 = "images_emotion/batch_3"
emotion_dir_batch4 = "images_emotion/batch_4"
emotion_dir_batch5 = "images_emotion/batch_5"

print(get_sorted_files(emotion_dir_batch3))  # Prints a sorted list of image file paths


In [ ]:
def run_example(task_prompt, image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer



In [ ]:
import json
import re
# Define the JSON file name
#json_filename = "response_Flo_action_1.json"

for i in range(1, 6):
  json_filename_action = f"response_action_LLaVa_{i}.json"
  json_filename_emotion = f"response_emotion_LLaVa_{i}.json"
  # Create an empty JSON structure
  data = {}

  # Write the empty structure to the file
  with open(json_filename_action, "w") as file:
      json.dump(data, file, indent=4)
  with open(json_filename_emotion, "w") as file:
      json.dump(data, file, indent=4)

  print(f"{json_filename_action} created successfully!")
  print(f"{json_filename_emotion} created successfully!")

Answer retrieval with direct questioning


In [ ]:
# action classes
jobs = [
        "Operating heavy machinery",
                    "Lifting materials",
                    "Measuring and marking surfaces",
                    "Mixing cement",
                    "Collaborating in teams",
                    "Using hand tools",
                    "Inspecting completed work",
                    "Communicating with supervisors",
                    "Following safety protocols",
                    "Taking breaks/resting"
        ]

# emotion classes
emotions = [
        "Focused",
        "Determined",
        "Tired",
        "Alert",
        "Satisfied",
        "Anxious",
        "Proud",
        "Frustrated",
        "Cooperative",
        "Relieved"
        ]

# data retrieval
def image_process(image_dir, json_filename, category):
  image_files = get_sorted_files(image_dir)
  data = {}
  for raw_image in image_files:
      print(f"\n Current process image: {raw_image}  \n")
      image = Image.open(raw_image)
      answer_data_for_each = []
      for j in category[1]:
        task_prompt = f'Does this {category[0]} of "{j}" appear in the photo? Answer only "yes" or "no". Do not explain or add any other words."'
        results = run_example(task_prompt, image=image)
        response_text = list(results.values())[0]
        answer_data_for_each.append(response_text)

        print(results)

        del results, response_text
        torch.cuda.empty_cache()

      print(answer_data_for_each)

      data[raw_image] = answer_data_for_each
      with open(json_filename, "w") as file:
          json.dump(data, file, indent=4)

      del answer_data_for_each

  del data
  torch.cuda.empty_cache()


**Get data for action recognition**


In [ ]:
image_process(image_dir = image_dir_batch1, json_filename="response_Flo_action_1.json", category=["action",jobs])

In [ ]:
image_process(image_dir = image_dir_batch2, json_filename="response_Flo_action_2.json", category=["action",jobs])

In [ ]:
image_process(image_dir=image_dir_batch3, json_filename="response_Flo_action_3.json", category=["action",jobs])

In [ ]:
image_process(image_dir=image_dir_batch4, json_filename="response_Flo_action_4.json", category=["action",jobs])

In [ ]:
image_process(image_dir=image_dir_batch5, json_filename="response_Flo_action_5.json", category=["action",jobs])

**Get data for emotion recognition**

In [ ]:
image_process(image_dir = emotion_dir_batch1, json_filename="response_Flo_emotion_1.json", category=["emotion",emotions])

In [ ]:
image_process(image_dir = emotion_dir_batch2, json_filename="response_Flo_emotion_2.json", category=["emotion",emotions])

In [ ]:
image_process(image_dir=emotion_dir_batch3, json_filename="response_Flo_emotion_3.json", category=["emotion",emotions])

In [ ]:
image_process(image_dir=emotion_dir_batch4, json_filename="response_Flo_emotion_4.json", category=["emotion",emotions])

In [ ]:
image_process(image_dir=emotion_dir_batch5, json_filename="response_Flo_emotion_5.json", category=["emotion",emotions])